# <font color=red>DATA GATHERING II: MUSIC GENRES AND SUBGENRES</font>

In [339]:
import pandas as pd
import numpy as np
import time
import math
import tqdm
import warnings
warnings.filterwarnings('ignore')

## <font color=blue>1) Genres and subgenres</font>

https://www.musicgenreslist.com/ + others in Musicbrainz - total: 927 into 14 subgenres:

- Blues
- Classical
- Country
- Electronic
- Folk
- Hip Hop
- Jazz
- Latin
- Pop
- Punk
- Rythm & Blues (R&B) / Soul
- Rock
- World (local music genres from specific regions of the world)
- Others (This category contains all the subgenres I haven't been able to classify in the previous categories)

According to Musicbrainz's Genre description in https://wiki.musicbrainz.org/Genre:

"Genres are currently supported in MusicBrainz as part of the tag system.

Some tags (the ones in the genre list) are automatically read and presented as genres."

What we want for our visualization is to have, for each release, its main genre and eventually its subgenre. To do so, I have copied Musicbrainz's "genre list" into a csv file. There are 419 elements considered as genres by Musicbrainz but for our study we'll consider them as our subgenres.

Of course, I wasn't familiar with all the genres appearing in the list so, in order to classify those, I looked at their definition in wikipedia and chose the best main genre for them. If no definition was provided by wikipedia, I searched for them in Google and listened to a representative song in order to make a decision.

In [340]:
all_genres = pd.read_csv('Main_genre_list.csv', sep='\t', header=0, encoding='utf-8')
all_genres.head()

,Main_genre,subgenre
0,Blues,acoustic blues
1,Blues,african blues
2,Blues,blues
3,Blues,blues music
4,Blues,blues rock


As we read before, Musicbrainz's genre list (subgenre for us) is part of their tag system. Let's import the Musicbrainz's "tags" table and try to identify, from its elements, the ones that are genres.

In [341]:
tags = pd.read_csv('Musicbrainz/Tables_used/tags.txt',sep='\t', header=None, engine='c', usecols=[0,1])
tags.columns = ['tag_id','tag_name']
tags.head()

,tag_id,tag_name
0,95,finnish
1,23,slovak
2,801,iowa
3,4,groundbreaking
4,130,taiwanese


In [342]:
#How many tags are there?
tags['tag_id'].nunique()

86806

In [343]:
#What do the tags look like?
tags.tag_name.value_counts()

rock                              2
alternative rock                  2
music box                         2
l'importante è finire             2
punk rock                         2
post punk                         2
enigmatic                         2
field recordings                  2
song writing                      2
acustica uach                     2
post rock                         2
down tempo                        2
vintage synthesizers              2
la escena                         2
alt rock                          2
campus miraflores                 2
rock music                        2
new age music                     2
tom pomposello                    2
pop punk                          2
ur so fail                        2
concept album                     2
new zealand                       2
rhythm & blues                    2
kitaro                            2
melanie music                     2
tangerine dream                   2
herb recordings             

As we can see, the tags list contains the genres but also other (more subjective) expressions that some users have chosen as representative for the music entity. 

We will add columns to this tags dataframe to distinguish which of them are actually genres/subgenres. As we will do the matching by tag_name, we have to format the tag_names as the ones in all_genres: without punctuation and in lower case.

In [344]:
#We first normalize in lower case the tag_names:
tags['tag_name'] = tags['tag_name'].str.lower()

In [345]:
#We replace the punctuation with a space:
tags['tag_name'] = tags['tag_name'].str.replace('#!?()*-%"/\,<>:$@.',' ')
#We remove leading & trainling spaces:
tags['tag_name'] = tags['tag_name'].str.strip()

In [346]:
#And now we can do the merging:
tags_genres = pd.merge(tags, all_genres, how='left', left_on='tag_name', right_on='subgenre')
tags_genres.head()

,tag_id,tag_name,Main_genre,subgenre
0,95,finnish,NaN,NaN
1,23,slovak,NaN,NaN
2,801,iowa,NaN,NaN
3,4,groundbreaking,NaN,NaN
4,130,taiwanese,NaN,NaN


In [347]:
#How many subgenres did we identify?
pd.notna(tags_genres['Main_genre']).value_counts()

False    86021
True       785
Name: Main_genre, dtype: int64

In [348]:
#What kind of tag_names haven't been associated with a Main genre?
tags_genres[tags_genres['Main_genre'].isnull()]

,tag_id,tag_name,Main_genre,subgenre
0,95,finnish,NaN,NaN
1,23,slovak,NaN,NaN
2,801,iowa,NaN,NaN
3,4,groundbreaking,NaN,NaN
4,130,taiwanese,NaN,NaN
5,134,thai,NaN,NaN
6,154,war,NaN,NaN
7,52,netlabel,NaN,NaN
8,101,cotm,NaN,NaN
9,82,punkrock,NaN,NaN


As we can see above, some of the tags that don't have a Main genre associated could be easily classified (for instance: "punkrock", or "dark metal"). 

Those tag names are not considered as a subgenre by Musicbrainz but they do provide us with some information about the release main genre. We will consider them as subgenre and identify their main genre.

What I will do now is to retrieve more information about these genreless tag_names in order to be able to classiffy them:

In [349]:
#Creating a specific dataframe for them:
genreless = tags_genres[pd.notna(tags_genres.tag_name) & pd.isnull(tags_genres.Main_genre)]
genreless.head()

,tag_id,tag_name,Main_genre,subgenre
0,95,finnish,NaN,NaN
1,23,slovak,NaN,NaN
2,801,iowa,NaN,NaN
3,4,groundbreaking,NaN,NaN
4,130,taiwanese,NaN,NaN


In [350]:
#We create new columns to retrieve some information about the content of each tag:
genreless['Blues'] = np.nan
genreless['Classical'] = np.nan
genreless['Country'] = np.nan
genreless['Electronic'] = np.nan
genreless['Folk'] = np.nan
genreless['Hip_Hop'] = np.nan
genreless['Jazz'] = np.nan
genreless['Latin'] = np.nan
genreless['Pop'] = np.nan
genreless['Punk'] = np.nan
genreless['RB'] = np.nan
genreless['Rock'] = np.nan
genreless.head()

,tag_id,tag_name,Main_genre,subgenre,Blues,Classical,Country,Electronic,Folk,Hip_Hop,Jazz,Latin,Pop,Punk,RB,Rock
0,95,finnish,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,23,slovak,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,801,iowa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,groundbreaking,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,130,taiwanese,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [351]:
#We create a column tag_name_clean where the text is formatted (remove punctuation, concatenate all words):
punctuation = ['#','!','?','(',')','*','-','%',' ',',',"'",'.','"','/','<','>',':','_']
genreless['tag_name_clean'] = genreless['tag_name'].apply(lambda x: ''.join(c for c in x if c not in punctuation))
genreless.head()

,tag_id,tag_name,Main_genre,subgenre,Blues,Classical,Country,Electronic,Folk,Hip_Hop,Jazz,Latin,Pop,Punk,RB,Rock,tag_name_clean
0,95,finnish,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,finnish
1,23,slovak,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,slovak
2,801,iowa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,iowa
3,4,groundbreaking,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,groundbreaking
4,130,taiwanese,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,taiwanese


In [352]:
#We remove the punctuation (except spaces) for the tag name too:
punctuation2 = ['#','!','?','(',')','*','-','%',',',"'",'.','"','/','<','>',':','_']
genreless['tag_name'] = genreless['tag_name'].apply(lambda x: ''.join(c for c in x if c not in punctuation2))
genreless.head()

,tag_id,tag_name,Main_genre,subgenre,Blues,Classical,Country,Electronic,Folk,Hip_Hop,Jazz,Latin,Pop,Punk,RB,Rock,tag_name_clean
0,95,finnish,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,finnish
1,23,slovak,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,slovak
2,801,iowa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,iowa
3,4,groundbreaking,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,groundbreaking
4,130,taiwanese,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,taiwanese


In [353]:
genreless[genreless['tag_id'] == 23291]

,tag_id,tag_name,Main_genre,subgenre,Blues,Classical,Country,Electronic,Folk,Hip_Hop,Jazz,Latin,Pop,Punk,RB,Rock,tag_name_clean
7723,23291,reggaetonlatinhiphop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,reggaetonlatinhiphop


In [354]:
#We create a pattern of words that could be associated with each genre:
Blues = 'blues'
Classical = 'classical|symphony|orchestra|stringquartet|pianist|opera|soprano|symph'
Country = 'country'
Electronic = 'electronic|electr|house|techno'
Folk = 'folk'
Hip_Hop = 'hiphop|rap|gangsta'
Jazz = 'jazz|jamband'
Latin = 'latin|reggaeton'
Pop = 'pop'
Punk = 'punk'
RB = 'rhythmandblues|rythmandblues|R&B'
Rock = 'rock|metal'

In [355]:
#And now we fill each genre column by searching if the column tag_name_clean contains the patterns:
genreless.Blues = np.where(genreless.tag_name_clean.str.contains(Blues), 'Blues', np.nan)
genreless.Classical = np.where(genreless.tag_name_clean.str.contains(Classical), 'Classical', np.nan)
genreless.Country = np.where(genreless.tag_name_clean.str.contains(Country), 'Country', np.nan)
genreless.Electronic = np.where(genreless.tag_name_clean.str.contains(Electronic), 'Electronic', np.nan)
genreless.Folk = np.where(genreless.tag_name_clean.str.contains(Folk), 'Folk', np.nan)
genreless.Hip_Hop = np.where(genreless.tag_name_clean.str.contains(Hip_Hop), 'Hip Hop', np.nan)
genreless.Jazz = np.where(genreless.tag_name_clean.str.contains(Jazz), 'Jazz', np.nan)
genreless.Latin = np.where(genreless.tag_name_clean.str.contains(Latin), 'Latin', np.nan)
genreless.Pop = np.where(genreless.tag_name_clean.str.contains(Pop), 'Pop', np.nan)
genreless.Punk = np.where(genreless.tag_name_clean.str.contains(Punk), 'Punk', np.nan)
genreless.RB = np.where(genreless.tag_name_clean.str.contains(RB), 'RB', np.nan)
genreless.Rock = np.where(genreless.tag_name_clean.str.contains(Rock), 'Rock', np.nan)

In [356]:
genreless.head()

,tag_id,tag_name,Main_genre,subgenre,Blues,Classical,Country,Electronic,Folk,Hip_Hop,Jazz,Latin,Pop,Punk,RB,Rock,tag_name_clean
0,95,finnish,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,finnish
1,23,slovak,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,slovak
2,801,iowa,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,iowa
3,4,groundbreaking,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,groundbreaking
4,130,taiwanese,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,taiwanese


In [357]:
genreless.replace('nan', np.nan, inplace=True)
genreless.head()

,tag_id,tag_name,Main_genre,subgenre,Blues,Classical,Country,Electronic,Folk,Hip_Hop,Jazz,Latin,Pop,Punk,RB,Rock,tag_name_clean
0,95,finnish,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,finnish
1,23,slovak,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,slovak
2,801,iowa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,iowa
3,4,groundbreaking,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,groundbreaking
4,130,taiwanese,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,taiwanese


What we want now, is to identify the tag_name which can contain more than 1 Main genre (e.g: "poprock"), and decide which is the main genre for them.

In [358]:
#We create a column "genre_counts" that sums the number of genres identified for each tag_name:
genreless['genre_counts'] = genreless.iloc[:,4:16].notnull().sum(axis=1)
genreless.head()

,tag_id,tag_name,Main_genre,subgenre,Blues,Classical,Country,Electronic,Folk,Hip_Hop,Jazz,Latin,Pop,Punk,RB,Rock,tag_name_clean,genre_counts
0,95,finnish,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,finnish,0
1,23,slovak,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,slovak,0
2,801,iowa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,iowa,0
3,4,groundbreaking,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,groundbreaking,0
4,130,taiwanese,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,taiwanese,0


In [359]:
#We gather all the genres in a new column:
start = time.time()

genreless['genres'] = np.nan
genreless.reset_index(drop=True, inplace=True)

for i in tqdm.tqdm(range(len(genreless))):
    if genreless['genre_counts'][i] != 0:
        a = genreless.loc[i,"Blues":"Rock"].notna()
        z = [i for i, x in enumerate(a) if x]
        genreless['genres'][i] = str(a[z].index.values)
    else:
        pass

end = time.time()
print((end-start)/60)

100%|██████████| 86019/86019 [14:52<00:00, 96.41it/s]  

14.871390748023988


In [360]:
#We can now get rid of the intermediary columns:
genreless.drop(labels=['subgenre','Blues', 'Classical', 'Country',
       'Electronic', 'Folk', 'Hip_Hop', 'Jazz', 'Latin', 'Pop',
       'Punk', 'RB', 'Rock', 'tag_name_clean'], axis=1, inplace=True)
genreless.head()

,tag_id,tag_name,Main_genre,genre_counts,genres
0,95,finnish,NaN,0,NaN
1,23,slovak,NaN,0,NaN
2,801,iowa,NaN,0,NaN
3,4,groundbreaking,NaN,0,NaN
4,130,taiwanese,NaN,0,NaN


In [361]:
#We can fill the main genre column for the ones that have just 1 genre identified:
genreless.Main_genre = np.where(genreless.genre_counts.isin([1]), genreless.genres,genreless.Main_genre )

In [362]:
#How many did we identify?
genreless.Main_genre.isnull().value_counts()

True     76535
False     9484
Name: Main_genre, dtype: int64

Not bad: we were able to retrieve the Main genre for 9.300 tags via this technique.

What we want now is to analyze the cases where there is more than one main genre identified:

In [363]:
genreless[genreless['genre_counts'] >1].head(100)

,tag_id,tag_name,Main_genre,genre_counts,genres
9,82,punkrock,NaN,2,['Punk' 'Rock']
13,52611,electro justice rock bbc one madeon remix daft...,NaN,3,['Electronic' 'Punk' 'Rock']
247,58451,echo park echopark rock pop rockpop guildford ...,NaN,2,['Pop' 'Rock']
445,729,popjazz,NaN,2,['Jazz' 'Pop']
563,898,irish folk rock,NaN,2,['Folk' 'Rock']
612,31371,popunk,NaN,2,['Pop' 'Punk']
661,1055,jazz metal,NaN,2,['Jazz' 'Rock']
676,1083,piano pop rock,NaN,2,['Pop' 'Rock']
680,1089,neoclassical metal,NaN,2,['Classical' 'Rock']
692,1111,electro rock,NaN,2,['Electronic' 'Rock']


#### Establishing dominant genres: 

In order to classify the tags that have been associated with more than one Main genre, we need to use some criteria. From my perspective, I think there are some Main genres that are dominant against others.

Again, music genre is something that can be very subjective in some cases: some people would consider The Beattles as a rock band, while I personally think they produced Pop music (maybe PopRock, but definitely not Rock music as I see it). 

As this project is done by myself, even if I try to be as objective as possible, I need to input my personal criteria and here they are:

 - If a tag has the genre "Electronic" associated, I consider it as Electronic music. 
 - If a tag isn't associated with Electronic music but with Punk music, I consider it as Punk music.
 - If a tag isn't included in the above and has the genre Pop in it, I consider it as Pop.
 - If a tag isn't included in the above and has the genre Rock in it, I consider it as Rock.
 - If a tag isn't included in the above and has the genre Hip Hop in it, I consider it as Hip Hop.

However, I will use this criteria only if the number of Main genres identified are two. I think the cases where there are more than 2 Main genres identified are probably incorrect tags (like, for instance "bossa-nova latin world pop folk jazz flamenco").

In [364]:
#We drop the rows for which we didn't retrieve any genre at all:
genreless.dropna(subset=['genres'], axis=0, inplace=True)

In [365]:
#We drop also the rows for whose the tag count is greater than 2:
genreless.drop(genreless[genreless['genre_counts'] > 2].index, inplace=True)

In [366]:
start = time.time()

#Filling the Main_genre column for our multiplt-tagged rows:

genreless.reset_index(drop=True, inplace=True)

for i in tqdm.tqdm(range(len(genreless))):
    if genreless['genre_counts'][i] == 2 and 'Electronic' in genreless['genres'][i]:
        genreless['Main_genre'][i] = 'Electronic'
    elif genreless['genre_counts'][i] == 2 and 'Punk' in genreless['genres'][i] and pd.isnull(genreless['Main_genre'][i]):
        genreless['Main_genre'][i] = 'Punk'
    elif genreless['genre_counts'][i] == 2 and 'Pop' in genreless['genres'][i] and pd.isnull(genreless['Main_genre'][i]):
        genreless['Main_genre'][i] = 'Pop'       
    elif genreless['genre_counts'][i] == 2 and 'Rock' in genreless['genres'][i] and pd.isnull(genreless['Main_genre'][i]):
        genreless['Main_genre'][i] = 'Rock'
    elif genreless['genre_counts'][i] == 2 and 'Hip_Hop' in genreless['genres'][i] and pd.isnull(genreless['Main_genre'][i]):
        genreless['Main_genre'][i] = 'Hip Hop'
    elif genreless['genre_counts'][i] == 2 and 'Jazz' in genreless['genres'][i] and pd.isnull(genreless['Main_genre'][i]):
        genreless['Main_genre'][i] = 'Jazz'
    elif genreless['genre_counts'][i] == 2 and 'Folk' in genreless['genres'][i] and pd.isnull(genreless['Main_genre'][i]):
        genreless['Main_genre'][i] = 'Folk'
    elif genreless['genre_counts'][i] == 2 and 'Blues' in genreless['genres'][i] and pd.isnull(genreless['Main_genre'][i]):
        genreless['Main_genre'][i] = 'Blues'
    elif genreless['genre_counts'][i] == 2 and 'Latin' in genreless['genres'][i] and pd.isnull(genreless['Main_genre'][i]):
        genreless['Main_genre'][i] = 'Latin'
    elif genreless['genre_counts'][i] == 2 and 'Classical' in genreless['genres'][i] and pd.isnull(genreless['Main_genre'][i]):
        genreless['Main_genre'][i] = 'Classical'
    else:
        pass

end = time.time()
print((end-start)/60)

100%|██████████| 11305/11305 [02:22<00:00, 79.18it/s] 

2.3797427932421367


In [367]:
#We remove the punctuation in Main_genre:
genreless['Main_genre'] = genreless['Main_genre'].str.strip('[]').str.strip("'_")
genreless.head()

,tag_id,tag_name,Main_genre,genre_counts,genres
0,82,punkrock,Punk,2,['Punk' 'Rock']
1,137,dark metal,Rock,1,['Rock']
2,34257,jazz blaxploitation,Jazz,1,['Jazz']
3,142,hardcore metal,Rock,1,['Rock']
4,33903,thrash death metal,Rock,1,['Rock']


In [368]:
#We delete the useless columns:
genreless.drop(labels=['genre_counts', 'genres'], axis=1, inplace=True)

In [369]:
#How many did we identify this time?
genreless.Main_genre.isnull().value_counts()

False    11305
Name: Main_genre, dtype: int64

We have identified an extra 1821 tag names in this last step. We are now ready to input this information into our tags_genres dataframe: 

In [370]:
#We first drop the Null values in Main_genre (those will be in genreless):
tags_genres.dropna(subset=['Main_genre'], axis=0, inplace=True)
#And the column subgenre which is not useful anymore:
tags_genres.drop(labels=['subgenre'], axis=1, inplace=True)

In [371]:
#Do the merging:
tags_all = pd.concat([tags_genres, genreless], ignore_index=True)
tags_all.head()

,tag_id,tag_name,Main_genre
0,24,digital hardcore,Electronic
1,28,raggacore,Electronic
2,79,techstep,Electronic
3,30,dubstep,Electronic
4,122,visual kei,Rock


In [372]:
#How many rows did we have without Main genre?
tags_all.Main_genre.isnull().value_counts()

False    12090
Name: Main_genre, dtype: int64

So we have been able to identify the Main genre for 11.954 tags in total: this will be very useful in the next steps.

## <font color=blue>2) Release genre</font>

### Data from Musicbrainz.org

Musicbrainz provides a table with all the release groups which have been tagged by their users. What we'll do next, is to retrieve those tags and select the ones that are part of the genres list.

In [373]:
#We import our main dataframe from the previous notebook:
df = pd.read_csv('Dataframe_with_origin.csv', sep='\t', header=0, encoding='utf-8')
df.head()

,release_id,release_group,group_id,release_year,artist_id,artist_mbid,credit_id,artist_name,area_id,area_name,ISO_code,ISO_country,lat,long
0,2265346,Le 1,2042812,2018-01-01,1720981.0,a69efb5f-0b28-4328-8ff0-44d8d6f39755,2291833,TedeuzeM,68613.0,Aix-en-Provence,NaN,FR,46.0000,2.0000
1,1772538,devil jokes,1656147,2016-01-01,1363025.0,c941ad72-8b13-4940-8d99-0ed9becad2d7,1653884,yzome,9655.0,Seattle,US-WA,US,47.0417,-122.8958
2,1247979,!,1234953,2009-01-01,834659.0,9d02b2a1-c9a7-46aa-8674-adf38c44d81a,874079,Gatuzo,53.0,Croatia,HR,HR,45.1667,15.5000
3,1571374,!,1497879,2010-01-01,674029.0,27a3d370-5430-42c0-8de4-1a7635d781b2,1440641,S.K.A.Y.,219.0,Ukraine,UA,UA,49.0000,32.0000
4,1528674,!,1463719,2014-01-01,491638.0,ed962474-bb85-47f9-b108-073184f09bc8,491638,Rusko,222.0,United States,US,US,38.0000,-97.0000


In [374]:
release_groups = pd.read_csv('Musicbrainz/Tables_used/release_group.txt',sep='\t', header=None, engine='c', usecols=[0,1,2])
release_groups.columns = ['group_id','group_mbid','release_group_name']
release_groups.head()

,group_id,group_mbid,release_group_name
0,1964563,f59da930-70ba-4992-a346-7ed2d8e3cda8,Wande
1,12,2b10653e-655d-34fe-9db4-77242d817a17,Chore of Enchantment
2,13,0eac6659-d590-3eb7-8c13-ed8b3fdf4ef7,The Inevitable
3,28,c554da1a-c1aa-30c3-b0bb-44b1b837de33,Piece and Love
4,60,06729175-db17-3443-add7-921739a92762,Ultimate Alternative Wavers


In [375]:
release_groups['group_id'].nunique()

1745126

In [376]:
len(release_groups)

1745126

In [377]:
group_tag = pd.read_csv('Musicbrainz/Tables_used/release_group_tag.txt',sep='\t', header=None, engine='c', usecols=[0,1,2])
group_tag.columns = ['group_id','tag_id','tag_counts']
group_tag.head()

,group_id,tag_id,tag_counts
0,93688,150,1
1,906692,1371,1
2,906692,6948,1
3,617615,11,1
4,617615,545,1


In [378]:
#We can now merge the release groups with the tag ids and tag counts:
Table = pd.merge(release_groups, group_tag, how='left', on='group_id')
Table.head()

,group_id,group_mbid,release_group_name,tag_id,tag_counts
0,1964563,f59da930-70ba-4992-a346-7ed2d8e3cda8,Wande,NaN,NaN
1,12,2b10653e-655d-34fe-9db4-77242d817a17,Chore of Enchantment,41017.0,2.0
2,13,0eac6659-d590-3eb7-8c13-ed8b3fdf4ef7,The Inevitable,1053.0,2.0
3,13,0eac6659-d590-3eb7-8c13-ed8b3fdf4ef7,The Inevitable,1230.0,1.0
4,13,0eac6659-d590-3eb7-8c13-ed8b3fdf4ef7,The Inevitable,71.0,3.0


In [379]:
#And finally have our release groups associated with their genres:
release_group_genre = pd.merge(Table, tags_all, how='left', on='tag_id')
release_group_genre.head()

,group_id,group_mbid,release_group_name,tag_id,tag_counts,tag_name,Main_genre
0,1964563,f59da930-70ba-4992-a346-7ed2d8e3cda8,Wande,NaN,NaN,NaN,NaN
1,12,2b10653e-655d-34fe-9db4-77242d817a17,Chore of Enchantment,41017.0,2.0,alternativeindie rock,Rock
2,13,0eac6659-d590-3eb7-8c13-ed8b3fdf4ef7,The Inevitable,1053.0,2.0,swing,Jazz
3,13,0eac6659-d590-3eb7-8c13-ed8b3fdf4ef7,The Inevitable,1230.0,1.0,dixieland,Jazz
4,13,0eac6659-d590-3eb7-8c13-ed8b3fdf4ef7,The Inevitable,71.0,3.0,jazz,Jazz


Let's stop here for a while and check one of the releases that has several genre tags associated. Let's do this with one of the most popular releases of all times: the album "Thriller", by the king of Pop music: Michael Jackson. 

In [380]:
release_group_genre[release_group_genre['group_mbid']=='f32fab67-77dd-3937-addc-9062e28e4c37']

,group_id,group_mbid,release_group_name,tag_id,tag_counts,tag_name,Main_genre
1429052,61656,f32fab67-77dd-3937-addc-9062e28e4c37,Thriller,7282.0,2.0,NaN,NaN
1429053,61656,f32fab67-77dd-3937-addc-9062e28e4c37,Thriller,642.0,2.0,disco,Pop
1429054,61656,f32fab67-77dd-3937-addc-9062e28e4c37,Thriller,7935.0,1.0,NaN,NaN
1429055,61656,f32fab67-77dd-3937-addc-9062e28e4c37,Thriller,24521.0,0.0,80 s and 90 s pop,Pop
1429056,61656,f32fab67-77dd-3937-addc-9062e28e4c37,Thriller,1060.0,1.0,dance-pop,Electronic
1429057,61656,f32fab67-77dd-3937-addc-9062e28e4c37,Thriller,303.0,3.0,funk,Others
1429058,61656,f32fab67-77dd-3937-addc-9062e28e4c37,Thriller,11.0,0.0,electronic,Electronic
1429059,61656,f32fab67-77dd-3937-addc-9062e28e4c37,Thriller,41021.0,2.0,NaN,NaN
1429060,61656,f32fab67-77dd-3937-addc-9062e28e4c37,Thriller,76.0,1.0,dance,Electronic
1429061,61656,f32fab67-77dd-3937-addc-9062e28e4c37,Thriller,41027.0,3.0,contemporary r&b,R&B/Soul


As we can see, "Pop" is the most used tag for this group so we should keep it as the release's genre.

As music genre is a very subjective feature, in order to be as "objective" as possible, we'll take into consideration the majority of the votes to chose the subgenre and main genre of each release group.

To do so, we group the release_group_genre dataframe by Main_genre and number of tag counts and keep the top genre for each release group.

In [381]:
release_scores = pd.pivot_table(release_group_genre,values='tag_counts', index=['group_id', 'Main_genre'], aggfunc=np.sum, fill_value=0, margins=True)
release_scores.reset_index(level=['group_id','Main_genre'], inplace=True)
release_scores.head()

,group_id,Main_genre,tag_counts
0,2,Hip_Hop,2
1,4,Electronic,28
2,4,Pop,3
3,4,Rock,3
4,11,Folk,1


To avoid incorrect taggings, we will take into consideration only the tags that have more than one vote:

In [382]:
release_scores_filtered = release_scores[release_scores['tag_counts'] > 1]

In [383]:
#We sort by group_id and tag_counts:
release_scores_filtered.sort_values(['group_id','tag_counts'], ascending=[True,False], inplace=True)
release_scores_filtered.head()

,group_id,Main_genre,tag_counts
0,2,Hip_Hop,2
1,4,Electronic,28
2,4,Pop,3
3,4,Rock,3
5,11,Jazz,5


In [384]:
#And now we can drop the duplicate group_ids, keeping the top Main_genre:
release_scores_filtered.drop_duplicates(subset=['group_id'],keep='first', inplace=True)

What we want now is to combine our main dataframe with this new genre information we just retrieved:

In [385]:
#We merge both dataframes:
main_df = pd.merge(df, release_scores_filtered, how='left', on='group_id')
main_df.head()

,release_id,release_group,group_id,release_year,artist_id,artist_mbid,credit_id,artist_name,area_id,area_name,ISO_code,ISO_country,lat,long,Main_genre,tag_counts
0,2265346,Le 1,2042812,2018-01-01,1720981.0,a69efb5f-0b28-4328-8ff0-44d8d6f39755,2291833,TedeuzeM,68613.0,Aix-en-Provence,NaN,FR,46.0000,2.0000,NaN,NaN
1,1772538,devil jokes,1656147,2016-01-01,1363025.0,c941ad72-8b13-4940-8d99-0ed9becad2d7,1653884,yzome,9655.0,Seattle,US-WA,US,47.0417,-122.8958,NaN,NaN
2,1247979,!,1234953,2009-01-01,834659.0,9d02b2a1-c9a7-46aa-8674-adf38c44d81a,874079,Gatuzo,53.0,Croatia,HR,HR,45.1667,15.5000,NaN,NaN
3,1571374,!,1497879,2010-01-01,674029.0,27a3d370-5430-42c0-8de4-1a7635d781b2,1440641,S.K.A.Y.,219.0,Ukraine,UA,UA,49.0000,32.0000,NaN,NaN
4,1528674,!,1463719,2014-01-01,491638.0,ed962474-bb85-47f9-b108-073184f09bc8,491638,Rusko,222.0,United States,US,US,38.0000,-97.0000,NaN,NaN


In [386]:
len(main_df)

1280797

In [387]:
main_df['release_id'].nunique()

1280797

In [388]:
#For how many releases do we have the main genre now?
main_df.Main_genre.isnull().value_counts()

True     1198089
False      82708
Name: Main_genre, dtype: int64

In [389]:
main_df.columns

Index(['release_id', 'release_group', 'group_id', 'release_year', 'artist_id',
       'artist_mbid', 'credit_id', 'artist_name', 'area_id', 'area_name',
       'ISO_code', 'ISO_country', 'lat', 'long', 'Main_genre', 'tag_counts'],
      dtype='object')

In [390]:
#We export the retrieved releases into a dataframe, and the pending into another:
retrieved1 = main_df[main_df['Main_genre'].notnull()]
pending1 = main_df[main_df['Main_genre'].isnull()]
#And remove the columns related to genre in the pending1 dataframe:
pending1.drop(labels=['tag_counts', 'Main_genre'], axis=1, inplace=True)

So, according to the above results, we have for now the genre for only 82.708 releases, under a total of 1.280.797 (6% of our dataframe only).

## <font color=blue>3) Artist genre</font>

In order to retrieve more genres, the next step is retrieving the artists' genre (the same we did for the release groups), and add them to our main_df.

Note: by doing this, we are assuming that each band or artist always produces the same musical genre. This is not 100% always accurate (especially if we look at the subgenres). However in general, we can say that the majority of the bands/artists usually stay in the same musical line during their professional lives and they can be categorized into the same "Main genre". Again, this is an assumption that we need to make in order to retrieve more info for this project.

For that, we'll use first Musicbrainz's artist_tag table and we'll follow the same process we did before.

In [391]:
artist_tag = pd.read_csv('Musicbrainz/Tables_used/artist_tag.txt',sep='\t', header=None, engine='c', usecols=[0,1,2])
artist_tag.columns = ['artist_id','tag_id','tag_counts']
artist_tag.head()

,artist_id,tag_id,tag_counts
0,468800,29,2
1,522545,63294,1
2,31390,173,1
3,108404,271,1
4,108404,7,1


In [392]:
#We merge it with the tags_genres dataframe:
artist_tag_genre = pd.merge(artist_tag, tags_all, how='left', on='tag_id')
artist_tag_genre.head()

,artist_id,tag_id,tag_counts,tag_name,Main_genre
0,468800,29,2,progressive rock,Rock
1,522545,63294,1,NaN,NaN
2,31390,173,1,NaN,NaN
3,108404,271,1,hard rock,Rock
4,108404,7,1,rock,Rock


In [393]:
#We drop the artists that don't have a Main genre:
artist_tag_genre.dropna(subset=['Main_genre'], axis=0, inplace=True)

In [394]:
#We retrieve the artist name:
artists = pd.read_csv('Musicbrainz/Tables_used/artist.txt',sep='\t', header=None, engine='c', usecols=[0,2])
artists.columns = ['artist_id','artist_name']
artists.head()

,artist_id,artist_name
0,805192,WIK▲N
1,371203,Pete Moutso
2,273232,Zachary
3,101060,The Silhouettes
4,145773,Aric Leavitt


In [395]:
len(artists)

1476425

In [396]:
#We remove the vague artists in our main dataframe:
labels = ['[unknown]','[nature sounds]','[dialogue]','[christmas music]', '[no artist]', '[church chimes]','Various Artists','[language instruction]']
artists.drop(artists[artists['artist_name'].isin(labels)].index, axis=0, inplace=True)

In [397]:
#We merge it with the artist dataframe to see the names for each artist:
artist_genre = pd.merge(artists, artist_tag_genre, on='artist_id', how='left')
artist_genre.head()

,artist_id,artist_name,tag_id,tag_counts,tag_name,Main_genre
0,805192,WIK▲N,NaN,NaN,NaN,NaN
1,371203,Pete Moutso,NaN,NaN,NaN,NaN
2,273232,Zachary,NaN,NaN,NaN,NaN
3,101060,The Silhouettes,NaN,NaN,NaN,NaN
4,145773,Aric Leavitt,NaN,NaN,NaN,NaN


In [398]:
#We drop the artists that don't have any Main_genre associated:
artist_genre.dropna(subset=['Main_genre'], axis=0, inplace=True)

We follow the same scoring procedure that we did with the releases:

In [399]:
artist_scores = pd.pivot_table(artist_genre,values='tag_counts', index=['artist_id', 'Main_genre'], aggfunc=np.sum, fill_value=0, margins=True)
artist_scores.reset_index(level=['artist_id','Main_genre'], inplace=True)
artist_scores.head()

,artist_id,Main_genre,tag_counts
0,4,Electronic,16
1,4,Hip_Hop,1
2,4,Rock,0
3,6,Electronic,1
4,6,Jazz,1


To avoid incorrect taggings, we will take into consideration only the tags that have more than one vote:

In [400]:
artist_scores_filtered = artist_scores[artist_scores['tag_counts'] > 1]

In [401]:
#We sort by group_id and tag_counts:
artist_scores_filtered.sort_values(['artist_id','tag_counts'], ascending=[True,False], inplace=True)
artist_scores_filtered.head()

,artist_id,Main_genre,tag_counts
0,4,Electronic,16
5,6,Rock,4
6,9,Electronic,10
9,11,Jazz,3
15,17,Rock,11


In [402]:
#And now we can drop the duplicate group_ids, keeping the top Main_genre:
artist_scores_filtered.drop_duplicates(subset=['artist_id'],keep='first', inplace=True)

In [403]:
#We add this new information into our pending1 dataframe:
main_df2 = pd.merge(pending1, artist_scores_filtered, how='left', on='artist_id')
main_df2.head()

,release_id,release_group,group_id,release_year,artist_id,artist_mbid,credit_id,artist_name,area_id,area_name,ISO_code,ISO_country,lat,long,Main_genre,tag_counts
0,2265346,Le 1,2042812,2018-01-01,1.72098e+06,a69efb5f-0b28-4328-8ff0-44d8d6f39755,2291833,TedeuzeM,68613.0,Aix-en-Provence,NaN,FR,46.0000,2.0000,NaN,NaN
1,1772538,devil jokes,1656147,2016-01-01,1.36302e+06,c941ad72-8b13-4940-8d99-0ed9becad2d7,1653884,yzome,9655.0,Seattle,US-WA,US,47.0417,-122.8958,NaN,NaN
2,1247979,!,1234953,2009-01-01,834659,9d02b2a1-c9a7-46aa-8674-adf38c44d81a,874079,Gatuzo,53.0,Croatia,HR,HR,45.1667,15.5000,NaN,NaN
3,1571374,!,1497879,2010-01-01,674029,27a3d370-5430-42c0-8de4-1a7635d781b2,1440641,S.K.A.Y.,219.0,Ukraine,UA,UA,49.0000,32.0000,Rock,3.0
4,1528674,!,1463719,2014-01-01,491638,ed962474-bb85-47f9-b108-073184f09bc8,491638,Rusko,222.0,United States,US,US,38.0000,-97.0000,Electronic,2.0


In [404]:
main_df2.isnull().sum(axis=0)

release_id            0
release_group         3
group_id              0
release_year          0
artist_id            97
artist_mbid          97
credit_id             0
artist_name         102
area_id               0
area_name             0
ISO_code           7273
ISO_country          27
lat                   0
long                  0
Main_genre       963369
tag_counts       963369
dtype: int64

In [405]:
len(main_df2)

1198089

Not bad: we have now "only" 963.369 releases with no Main genre, so we have just retrieved the info for an extra 234.720 releases using the artists' information. In total, we have for now 317.428 releases with their genre information, so 25% of our Dataframe.

In [406]:
#We split the dataframe again:
retrieved2 = main_df2[main_df2['Main_genre'].notnull()]
pending2 = main_df2[main_df2['Main_genre'].isnull()]
#And remove the columns related to genre in the pending2 dataframe:
pending2.drop(labels=['tag_counts', 'Main_genre'], axis=1, inplace=True)

In [407]:
len(retrieved2)

234720

### Data from Wikidata Query with SPARQL

In [408]:
#Open the files and load them into dataframes with the same column names (to match with our main dataframe later):
musicians = pd.read_csv('wikidata/query_wikidata_musicians.csv',sep=',', encoding='utf-8', usecols=[3,4])
musicians.rename(columns={'genreLabel':'artist_genre','MusicBrainz_artist_ID':'artist_mbid'}, inplace=True)
singers = pd.read_csv('wikidata/query_wikidata_singers.csv',sep=',', encoding='utf-8', usecols=[3,4])
singers.rename(columns={'genreLabel':'artist_genre','MusicBrainz_artist_ID':'artist_mbid'}, inplace=True)
bands = pd.read_csv('wikidata/query_wikidata_bands.csv',sep=',', encoding='utf-8', usecols=[3,4])
bands.rename(columns={'genreLabel':'artist_genre','MusicBrainz_artist_ID':'artist_mbid'}, inplace=True)

In [409]:
#Now we can concatenate the 3 dataframes into one:
wiki_df = pd.concat([musicians, singers, bands])
wiki_df.head()

,artist_genre,artist_mbid
0,NaN,NaN
1,opera,b972f589-fb0e-474e-b64a-803b0364fa75
2,classical music,b972f589-fb0e-474e-b64a-803b0364fa75
3,symphony,b972f589-fb0e-474e-b64a-803b0364fa75
4,concerto,b972f589-fb0e-474e-b64a-803b0364fa75


In [410]:
#We merge the dataframe with the tags_genres to retrieve tag_id and Main_genre:
wiki_genres = pd.merge(wiki_df, tags_all, how='left', left_on='artist_genre', right_on='tag_name')
wiki_genres.head()

,artist_genre,artist_mbid,tag_id,tag_name,Main_genre
0,NaN,NaN,NaN,NaN,NaN
1,opera,b972f589-fb0e-474e-b64a-803b0364fa75,480.0,opera,Classical
2,classical music,b972f589-fb0e-474e-b64a-803b0364fa75,2092.0,classical music,Classical
3,classical music,b972f589-fb0e-474e-b64a-803b0364fa75,54585.0,classical music,Classical
4,symphony,b972f589-fb0e-474e-b64a-803b0364fa75,2806.0,symphony,Classical


In [411]:
#We drop the rows that don't have any artist_mbid (as we won't be able to match them):
wiki_genres.dropna(subset=['artist_mbid'], axis=0, inplace=True)

As some artists appear more than once (if they have more than one tag), we will have to score again the top Main genre. This time, however, as we don't have any tag_counts we will keep the top Main genre directly.

In [412]:
wiki_scores = pd.pivot_table(wiki_genres, index=['artist_mbid', 'Main_genre'], aggfunc='count')
wiki_scores.reset_index(level=['artist_mbid','Main_genre'], inplace=True)
wiki_scores.drop(labels=['tag_id', 'tag_name'], axis=1, inplace=True)
wiki_scores.head()

,artist_mbid,Main_genre,artist_genre
0,00010eb3-ebfe-4965-81ef-0ac64cd49fde,Latin,1
1,00034ede-a1f1-4219-be39-02f36853373e,World,1
2,0004537a-4b12-43eb-a023-04009e738d2e,Electronic,2
3,00050e90-e93a-4b06-b233-8899d437d201,Rock,2
4,00077d46-7b4a-4761-9eed-c7dd435fa5ff,Rock,2


In [413]:
#We sort by artist_id and artist_genre:
wiki_scores.sort_values(['artist_mbid','artist_genre'], ascending=[True,False], inplace=True)
wiki_scores.head()

,artist_mbid,Main_genre,artist_genre
0,00010eb3-ebfe-4965-81ef-0ac64cd49fde,Latin,1
1,00034ede-a1f1-4219-be39-02f36853373e,World,1
2,0004537a-4b12-43eb-a023-04009e738d2e,Electronic,2
3,00050e90-e93a-4b06-b233-8899d437d201,Rock,2
4,00077d46-7b4a-4761-9eed-c7dd435fa5ff,Rock,2


In [414]:
#And now we can drop the duplicate artist_ids, keeping the top Main_genre:
wiki_scores.drop_duplicates(subset=['artist_mbid'],keep='first', inplace=True)

In [415]:
#Now we can input this new information into our main dataframe:
main_df3 = pd.merge(pending2, wiki_scores, how='left', on='artist_mbid')
main_df3.drop(labels=['artist_genre'], axis=1, inplace=True)
main_df3.head()

,release_id,release_group,group_id,release_year,artist_id,artist_mbid,credit_id,artist_name,area_id,area_name,ISO_code,ISO_country,lat,long,Main_genre
0,2265346,Le 1,2042812,2018-01-01,1.72098e+06,a69efb5f-0b28-4328-8ff0-44d8d6f39755,2291833,TedeuzeM,68613.0,Aix-en-Provence,NaN,FR,46.0000,2.0000,NaN
1,1772538,devil jokes,1656147,2016-01-01,1.36302e+06,c941ad72-8b13-4940-8d99-0ed9becad2d7,1653884,yzome,9655.0,Seattle,US-WA,US,47.0417,-122.8958,NaN
2,1247979,!,1234953,2009-01-01,834659,9d02b2a1-c9a7-46aa-8674-adf38c44d81a,874079,Gatuzo,53.0,Croatia,HR,HR,45.1667,15.5000,NaN
3,1947276,! (LP/2017),1791666,2017-01-01,627929,e95d5542-401d-4319-9918-7cbbc507b758,627929,Chapelier Fou,73.0,France,FR,FR,46.0000,2.0000,NaN
4,164676,! (The Song Formerly Known As),230927,1998-01-01,12233,6b98b15d-a6a6-4190-9e15-56976f2c1068,12233,Regurgitator,13.0,Australia,AU,AU,-27.0000,133.0000,Rock


In [416]:
main_df3.isnull().sum(axis=0)

release_id            0
release_group         3
group_id              0
release_year          0
artist_id            97
artist_mbid          97
credit_id             0
artist_name         102
area_id               0
area_name             0
ISO_code           6061
ISO_country          27
lat                   0
long                  0
Main_genre       762837
dtype: int64

We have now 762.837 releases with no Main genre, so we have just retrieved the info for an extra 200.532 releases using the artists' information. In total, we have for now 517.960 releases with their genre information, so 40% of our Dataframe.

In [417]:
#We split again the dataframe in two, and keep retrieving:
retrieved3 = main_df3[main_df3['Main_genre'].notnull()]
pending3 = main_df3[main_df3['Main_genre'].isnull()]
#And remove the columns related to genre in the pending2 dataframe:
pending3.drop(labels=['Main_genre'], axis=1, inplace=True)

## Data from 1 Million Songs Dataset

In [418]:
#We open the file where some tracks have their genre associated:
tracks = pd.read_csv('1M_songs/msd_tagtraum_cd2c.csv', header=0, usecols = [0,1])
tracks.head()

,track_id,majority_genre
0,TRAAAAK128F9318786,Rock
1,TRAAAAW128F429D538,Rap
2,TRAAADJ128F4287B47,Rock
3,TRAAADZ128F9348C2E,Latin
4,TRAAAED128E0783FAB,Jazz


As we can see, these track have already a majority genre established. Which ones are there?

In [419]:
tracks.majority_genre.value_counts()

Rock          75013
Electronic    21865
Jazz          14700
Pop           12967
Rap           11001
RnB            9811
Metal          9224
Country        8983
Reggae         7970
Blues          6219
Folk           4188
Punk           3275
Latin          3113
World          1919
New Age        1153
Name: majority_genre, dtype: int64

Luckily, their groups are very similar to our Main genres so we'll just need to make a few changes of names in order for them to fit our classification:

- "Rap" will be changed to "Hip Hop"
- "RnB" will be changed to "R&B/Soul"
- "Metal" will be changed to "Rock"
- "Reggae" will be changed to "World"
- "New Age" will be changed to "Others"

In [420]:
tracks.replace({'Rap':'Hip Hop', 'RnB':'R&B/Soul', 'Metal':'Rock', 'Reggae': 'World', 'New Age':'Others'}, inplace=True)
tracks.majority_genre.value_counts()

Rock          84237
Electronic    21865
Jazz          14700
Pop           12967
Hip Hop       11001
World          9889
R&B/Soul       9811
Country        8983
Blues          6219
Folk           4188
Punk           3275
Latin          3113
Others         1153
Name: majority_genre, dtype: int64

In [421]:
#We open the file where we can match track_id and artist_mbid:
tracks_metadata = pd.read_csv('1M_songs/track_metadata.csv', header=0, usecols = [0,5])
tracks_metadata.head()

,track_id,artist_mbid
0,TRMMMYQ128F932D901,357ff05d-848a-44cf-b608-cb34b5701ae5
1,TRMMMKD128F425225D,8d7ef530-a6fd-4f8f-b2e2-74aec765e0f9
2,TRMMMRX128F93187D9,3d403d44-36ce-465c-ad43-ae877e65adc4
3,TRMMMCH128F425532C,12be7648-7094-495f-90e6-df4189d68615
4,TRMMMWA128F426B589,NaN


In [422]:
#We drop the rows with no value in artist_mbid:
tracks_metadata.dropna(subset=['artist_mbid'], axis=0, inplace=True)

In [423]:
#We merge the tracks dataframe with tracks metadata to retrieve the genre by artist:
artist_genre_1m = pd.merge(tracks, tracks_metadata, how='left', on='track_id')
artist_genre_1m.drop(labels=['track_id'], axis=1, inplace=True)
artist_genre_1m.head()

,majority_genre,artist_mbid
0,Rock,6ae6a016-91d7-46cc-be7d-5e8e5d320c54
1,Hip Hop,e77e51a5-4761-45b3-9847-2051f811e366
2,Rock,3cf5a3be-25ef-4408-98fe-e66fee536be1
3,Latin,7a273984-edd9-4451-9c4d-39b38f05ebcd
4,Jazz,e0e9d279-37d5-4493-99b8-5a21309502f6


In [424]:
artist_genre_1m.duplicated(subset='artist_mbid').value_counts()

True     170381
False     21020
dtype: int64

There is more than one genre associated with each artist so, we will repeat our scoring procedure:

In [425]:
artist_genre_1m['count'] = 1
scores_1m = pd.pivot_table(artist_genre_1m, index=['artist_mbid', 'majority_genre'], aggfunc='count')
scores_1m.reset_index(level=['artist_mbid','majority_genre'], inplace=True)
scores_1m.head()

,artist_mbid,majority_genre,count
0,0002f649-8285-4a72-b847-b3854e1a449c,Rock,12
1,00034ede-a1f1-4219-be39-02f36853373e,Rock,11
2,0004537a-4b12-43eb-a023-04009e738d2e,Electronic,1
3,00077d46-7b4a-4761-9eed-c7dd435fa5ff,Rock,2
4,000842dd-08e9-485f-a9b6-8ada9f1c4a12,World,1


In [426]:
#We sort by artist_id and artist_genre:
scores_1m.sort_values(['artist_mbid','count'], ascending=[True,False], inplace=True)
scores_1m.head()

,artist_mbid,majority_genre,count
0,0002f649-8285-4a72-b847-b3854e1a449c,Rock,12
1,00034ede-a1f1-4219-be39-02f36853373e,Rock,11
2,0004537a-4b12-43eb-a023-04009e738d2e,Electronic,1
3,00077d46-7b4a-4761-9eed-c7dd435fa5ff,Rock,2
4,000842dd-08e9-485f-a9b6-8ada9f1c4a12,World,1


In [427]:
#And now we can drop the duplicate artist_ids, keeping the top Main_genre:
scores_1m.drop_duplicates(subset=['artist_mbid'],keep='first', inplace=True)
scores_1m.drop(labels=['count'], axis=1, inplace=True)

In [428]:
#Is there any null value in Main_genre?
scores_1m.majority_genre.isnull().value_counts()

False    21019
Name: majority_genre, dtype: int64

In [429]:
#And we can input this information into our pending3 dataframe:
main_df4 = pd.merge(pending3, scores_1m, how='left', on='artist_mbid')
#How many releases did we retrieve the genre for in this last step?
main_df4.majority_genre.isnull().value_counts()

True     711087
False     51750
Name: majority_genre, dtype: int64

In [430]:
main_df4.rename(columns={'majority_genre':'Main_genre'}, inplace=True)

We have now 711.087 releases with no Main genre, so we have just retrieved the info for an extra 51.750 releases using the 1 Million Songs data. In total, we have for now 569.710 releases with their genre information, so 44% of our Dataframe.

We'll split the dataframe in two like we did before, and step to the last part of this notebook.

In [444]:
#We split again the dataframe in two, and keep retrieving:
retrieved4 = main_df4[main_df4['Main_genre'].notnull()]
pending4 = main_df4[main_df4['Main_genre'].isnull()]

In [447]:
pending4.drop(labels=['Main_genre'], axis=1, inplace=True)

### Extend artist genre into all the dataframe:

The idea of this last stage is to verify if, for the same artist, some releases have a main genre and others don't (this could have happened in the first stage, when we retrieved the genre by release group).

In order to do it, we will concatenate all our dataframes retrieved and pending, and check it:

In [448]:
retrieved1.drop(labels=['tag_counts'], axis=1, inplace=True)
retrieved2.drop(labels=['tag_counts'], axis=1, inplace=True)

KeyError: "['tag_counts'] not found in axis"

In [449]:
#We can concatenate them:
main_df5 = pd.concat([retrieved1, retrieved2, retrieved3, retrieved4, pending4 ], ignore_index=True)
main_df5.head()

,ISO_code,ISO_country,Main_genre,area_id,area_name,artist_id,artist_mbid,artist_name,credit_id,group_id,lat,long,release_group,release_id,release_year
0,US,US,Electronic,222.0,United States,109013,f26c72d3-e52c-467b-b651-679c73d8e1a7,!!!,109013,150660,38.0000,-97.00,!!!,9236,2001-01-01
1,NL,NL,Electronic,150.0,Netherlands,45223,734fa82c-864e-468b-bee4-944cb4b1952b,Speedy J,45223,85169,52.5000,5.75,!ive,57031,1995-01-01
2,US,US,Rock,222.0,United States,21416,3ec17e85-9284-4f4c-8831-4e56c2354cdb,Reba McEntire,21416,438466,38.0000,-97.00,# 1's,78797,2005-01-01
3,SK,SK,Electronic,189.0,Slovakia,283685,e4b17c6c-2951-4513-a811-9ae7ad959c4a,Olga+Jozef,283685,690150,48.6667,19.50,#01,359380,1999-01-01
4,SK,SK,Electronic,189.0,Slovakia,283685,e4b17c6c-2951-4513-a811-9ae7ad959c4a,Olga+Jozef,283685,690151,48.6667,19.50,#02,359383,1999-01-01


In [450]:
len(main_df5)

1280797

Now we want to select all the artists that have a Main genre associated, and count the number of releases that they have for each genre (this will be also like the scoring technique we used previously)

In [451]:
copy_maindf = main_df5[['artist_id', 'Main_genre']].copy()
copy_maindf['count'] = 1

In [452]:
scores_maindf = pd.pivot_table(copy_maindf, index=['artist_id', 'Main_genre'], aggfunc='count')
scores_maindf.reset_index(level=['artist_id','Main_genre'], inplace=True)
scores_maindf.head()

,artist_id,Main_genre,count
0,4.0,Electronic,80
1,6.0,Rock,3
2,9.0,Electronic,22
3,10.0,Electronic,1
4,10.0,Rock,5


In [453]:
#We sort by artist_id and artist_genre:
scores_maindf.sort_values(['artist_id','count'], ascending=[True,False], inplace=True)
scores_maindf.head()

,artist_id,Main_genre,count
0,4.0,Electronic,80
1,6.0,Rock,3
2,9.0,Electronic,22
4,10.0,Rock,5
3,10.0,Electronic,1


In order to avoid incorrectly assigning a genre to an artist who only has 1 release associated with it, in this case we'll consider only the release_count by genre above 1:

In [454]:
scores_maindf.drop(scores_maindf[scores_maindf['count'].isin([1])].index, axis=0, inplace=True)

In [455]:
#And now we can drop the duplicate artist_ids, keeping the top Main_genre:
scores_maindf.drop_duplicates(subset=['artist_id'],keep='first', inplace=True)
scores_maindf.drop(labels=['count'], axis=1, inplace=True)

In [456]:
len(scores_maindf)

50204

In [457]:
#Finally, we can merge our pending4 dataframe with this last one, and see if we retrieved more info:
main_df6 = pd.merge(pending4, scores_maindf, how='left', on='artist_id')

In [459]:
main_df6.Main_genre.isnull().value_counts()

True     688556
False     22531
Name: Main_genre, dtype: int64

Thankfully, by applying this last strategy, we have identified the genre for and extra 22.531 releases, which means that we have now a total of 592.241 releases with their genre (46% of the dataset).

Now, we'll gather all the information retrieved in a single file, and the information pending in another file:

In [461]:
#We split again the dataframe in two, and keep retrieving:
retrieved5 = main_df6[main_df6['Main_genre'].notnull()]
pending5 = main_df6[main_df6['Main_genre'].isnull()]

In [462]:
#Now we can concatenate the 5 retrieved dataframes:
data_out = pd.concat([retrieved1, retrieved2, retrieved3, retrieved4, retrieved5 ], ignore_index=True)
data_out.head()

,release_id,release_group,group_id,release_year,artist_id,artist_mbid,credit_id,artist_name,area_id,area_name,ISO_code,ISO_country,lat,long,Main_genre
0,9236,!!!,150660,2001-01-01,109013,f26c72d3-e52c-467b-b651-679c73d8e1a7,109013,!!!,222.0,United States,US,US,38.0000,-97.00,Electronic
1,57031,!ive,85169,1995-01-01,45223,734fa82c-864e-468b-bee4-944cb4b1952b,45223,Speedy J,150.0,Netherlands,NL,NL,52.5000,5.75,Electronic
2,78797,# 1's,438466,2005-01-01,21416,3ec17e85-9284-4f4c-8831-4e56c2354cdb,21416,Reba McEntire,222.0,United States,US,US,38.0000,-97.00,Rock
3,359380,#01,690150,1999-01-01,283685,e4b17c6c-2951-4513-a811-9ae7ad959c4a,283685,Olga+Jozef,189.0,Slovakia,SK,SK,48.6667,19.50,Electronic
4,359383,#02,690151,1999-01-01,283685,e4b17c6c-2951-4513-a811-9ae7ad959c4a,283685,Olga+Jozef,189.0,Slovakia,SK,SK,48.6667,19.50,Electronic


In [463]:
#Do we have our 592.241 releases?
len(data_out)

592241

In [465]:
data_out.Main_genre.value_counts()

Rock          201285
Electronic    112812
Pop            78101
Jazz           39636
Classical      35261
Hip Hop        21546
Folk           14627
Country        14359
R&B/Soul       13187
Punk           13134
Others         12570
Latin          11485
Blues          11189
World           9909
Hip_Hop         3140
Name: Main_genre, dtype: int64

In [466]:
#It looks like we have 2 different names for Hip Hop:
data_out.Main_genre = np.where(data_out.Main_genre == 'Hip_Hop', 'Hip Hop',data_out.Main_genre)

In [467]:
#We export the dataframes:
data_out.to_csv('data_out.csv', sep='\t', index=False, encoding='utf-8')
pending5.to_csv('data_pending.csv', sep='\t', index=False, encoding='utf-8')